# Load in extracted features

In [182]:
# open the every file in a directory and print the first line of each file
import os
import sys
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN


In [148]:
# load in the features
features_train = np.array([])
for file in os.listdir('features_train'):
    with open('features_train/' + file) as f:
        df = pd.read_csv(f)
        features_train = np.column_stack((features_train, df.values)) if features_train.size else df.values

# load the labels
train = pd.read_csv('train.csv')
y_train = train['Genre']
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
# normalize the data
features_train = preprocessing.scale(features_train)

# Load in test Data


In [149]:
features_test = np.array([])
for file in os.listdir('features_test'):
    with open('features_test/' + file) as f:
        df = pd.read_csv(f)
        features_test = np.column_stack((features_test, df.values)) if features_test.size else df.values

# scale the test data
features_test = preprocessing.scale(features_test)


# Logistic Regression

In [154]:

logreg = LogisticRegression(max_iter=150)
logreg.fit(features_train, y_train)
predictions = logreg.predict(features_train)
# Calculate the accuracy
accuracy = accuracy_score(y_train, predictions)
print("Logistic Regression accuracy:", accuracy)


Logistic Regression accuracy: 0.8075


# Predict with Logistic Regression and save to CSV

In [155]:

# run the model on the test data
predictions = logreg.predict(features_test)
predictions = label_encoder.inverse_transform(predictions)
# save the predictions to a csv file
df = pd.DataFrame({'ID': os.listdir('test'), 'genre': predictions})
df.to_csv('predictions.csv', index=False)

# Neural Network

In [162]:


model = Sequential()
model.add(Dense(80, activation='relu', input_shape = (37,)))

model.add(Dense(128, activation='relu'))

model.add(Dense(36, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(features_train, y_train, epochs=20, batch_size=32)

loss, accuracy = model.evaluate(features_train, y_train)
print('Neural Network accuracy:', accuracy)

Epoch 1/20


25/25 [==============================] - 0s 1ms/step - loss: 2.8905 - accuracy: 0.2488
Epoch 2/20
25/25 [==============================] - 0s 957us/step - loss: 1.6132 - accuracy: 0.5075
Epoch 3/20
25/25 [==============================] - 0s 947us/step - loss: 1.2198 - accuracy: 0.5875
Epoch 4/20
25/25 [==============================] - 0s 958us/step - loss: 1.0601 - accuracy: 0.6350
Epoch 5/20
25/25 [==============================] - 0s 943us/step - loss: 0.9617 - accuracy: 0.6637
Epoch 6/20
25/25 [==============================] - 0s 1ms/step - loss: 0.8779 - accuracy: 0.6875
Epoch 7/20
25/25 [==============================] - 0s 1ms/step - loss: 0.8312 - accuracy: 0.7075
Epoch 8/20
25/25 [==============================] - 0s 979us/step - loss: 0.7801 - accuracy: 0.7250
Epoch 9/20
25/25 [==============================] - 0s 905us/step - loss: 0.7433 - accuracy: 0.7337
Epoch 10/20
25/25 [==============================] - 0s 1ms/step - loss: 0.7138 - accuracy: 0.7425
Epoch 11/20
25/25 

# Predict with Neural Network and save to CSV

In [163]:
predictions = model.predict(features_test)
predictions = np.argmax(predictions, axis=1)
predictions = label_encoder.inverse_transform(predictions)


df = pd.DataFrame({'ID': os.listdir('test'), 'genre': predictions})
df.to_csv('predictions.csv', index=False)

7/7 [==============================] - 0s 812us/step
